In [40]:
import requests
import json

# Replace with your actual API key
API_KEY = "6011ead8a3f8e2b361a6d9b4f34c9060b1cf18b05e441d9752"

# Endpoint to get recent user alerts
url = "https://tempstickapi.com/api/v1/user/notifications"

# Set up headers with correct X-API-KEY
headers = {
    "X-API-KEY": API_KEY,
    "Accept-Encoding": "gzip",
    "Accept": "*/*"
}

# Make the GET request
response = requests.get(url, headers=headers)

def fetch_all_alerts(max_pages=100):
    page = 0
    all_alerts = []

    while page < max_pages:
        url = f"{BASE_URL}?page={page}&items_per_page=100"
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        data = response.json()

        if data["type"] != "success":
            print(f"⚠️ API error: {data.get('message', 'Unknown error')}")
            break

        alerts = data["data"]["items"]
        if not alerts:
            break

        all_alerts.extend(alerts)
        print(f"Fetched page {page+1}, total alerts so far: {len(all_alerts)}")

        page += 1

    return all_alerts

if __name__ == "__main__":
    alerts = fetch_all_alerts()

    print(f"Total alerts fetched: {len(alerts)}\n")

    for alert in alerts:
        if alert["event_type"] == "temperature" and "above" in alert["event_message"]:
            message = alert["event_message"]
            try:
                temp_f = float(message.split("(")[1].split()[0])
                print(f"🔥 Alert: {message}")
                print(f"📅 Time: {alert['created']}")
                print(f"🆔 Sensor ID: {alert['sensor_id']}\n")
            except Exception as e:
                print(f"❌ Error parsing temp: {message} | {e}")

HTTPError: 404 Client Error: Not Found for url: https://tempstickapi.com/api/v1/?page=0&items_per_page=100

In [42]:
import requests
from dateutil import parser

API_KEY = "6011ead8a3f8e2b361a6d9b4f34c9060b1cf18b05e441d9752"
BASE_URL = "https://tempstickapi.com/api/v1/user/notifications"
HEADERS = {
    "X-API-KEY": API_KEY,
    "Accept-Encoding": "gzip",
    "Accept": "*/*"
}

def fetch_all_alerts(max_pages=100):
    page = 0
    all_alerts = []

    while page < max_pages:
        url = f"{BASE_URL}?page={page}&items_per_page=100"
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        data = response.json()

        if data["type"] != "success":
            print(f"⚠️ API error: {data.get('message', 'Unknown error')}")
            break

        alerts = data["data"]["items"]
        if not alerts:
            break

        all_alerts.extend(alerts)
        print(f"Fetched page {page+1}, total alerts so far: {len(all_alerts)}")

        page += 1

    return all_alerts

if __name__ == "__main__":
    alerts = fetch_all_alerts()

    print(f"Total alerts fetched: {len(alerts)}\n")

    for alert in alerts:
        if alert["event_type"] == "temperature" and "above" in alert["event_message"]:
            message = alert["event_message"]
            try:
                temp_f = float(message.split("(")[1].split()[0])
                print(f"🔥 Alert: {message}")
                print(f"📅 Time: {alert['created']}")
                print(f"🆔 Sensor ID: {alert['sensor_id']}\n")
            except Exception as e:
                print(f"❌ Error parsing temp: {message} | {e}")

Fetched page 1, total alerts so far: 100
Fetched page 2, total alerts so far: 200
Fetched page 3, total alerts so far: 300
Fetched page 4, total alerts so far: 400
Fetched page 5, total alerts so far: 500
Fetched page 6, total alerts so far: 600
Fetched page 7, total alerts so far: 700
Fetched page 8, total alerts so far: 800
Fetched page 9, total alerts so far: 900
Fetched page 10, total alerts so far: 1000
Fetched page 11, total alerts so far: 1100
Fetched page 12, total alerts so far: 1200
Fetched page 13, total alerts so far: 1300
Fetched page 14, total alerts so far: 1400
Fetched page 15, total alerts so far: 1500
Fetched page 16, total alerts so far: 1600
Fetched page 17, total alerts so far: 1700
Fetched page 18, total alerts so far: 1800
Fetched page 19, total alerts so far: 1900
Fetched page 20, total alerts so far: 2000
Fetched page 21, total alerts so far: 2100
Fetched page 22, total alerts so far: 2200
Fetched page 23, total alerts so far: 2300
Fetched page 24, total alerts

In [44]:
import os
import sqlite3
import requests
from datetime import datetime, timedelta, timezone

try:
    from zoneinfo import ZoneInfo
except ImportError:
    import pytz
    class PyTZZoneInfo:
        def __init__(self, key): self._tz = pytz.timezone(key)
        def __getattr__(self, name): return getattr(self._tz, name)
    ZoneInfo = PyTZZoneInfo

PACIFIC_TZ = ZoneInfo("America/Los_Angeles") if ZoneInfo else None

API_KEY = os.environ.get("API_KEY", "6011ead8a3f8e2b361a6d9b4f34c9060b1cf18b05e441d9752")
BASE_URL = os.environ.get("BASE_URL", "https://tempstickapi.com/api/v1")
HEADERS = {"X-API-KEY": API_KEY, "Accept": "application/json"}

DB_PATH = os.path.expanduser("~/projects/taft_projects/data_analysis/TempStick/data/ninthbrain_incidents.db")
ALERT_HISTORY_DAYS = 7

def safe_get_request(url, params=None):
    try:
        response = requests.get(url, headers=HEADERS, params=params, timeout=20)
        response.raise_for_status()
        if 'application/json' in response.headers.get('Content-Type', ''):
            return response.json()
    except Exception as e:
        print(f"Error requesting {url}: {e}")
    return None

def get_recent_alerts(sensor_id):
    url = f"{BASE_URL}/sensor/notifications/{sensor_id}"
    now = datetime.now(timezone.utc)
    params = {
        'startDate': (now - timedelta(days=ALERT_HISTORY_DAYS)).strftime('%Y-%m-%dT%H:%M:%SZ'),
        'endDate': now.strftime('%Y-%m-%dT%H:%M:%SZ')
    }
    response = safe_get_request(url, params)
    if response:
        return response.get('data', {}).get('items', [])
    return []

def summarize_alert_periods(alerts):
    grouped = []
    current = None
    alerts.sort(key=lambda x: x.get('created', ''))
    for alert in alerts:
        if not isinstance(alert, dict): continue
        msg, ts, typ = alert.get('event_message', ''), alert.get('created', ''), alert.get('event_type', '')
        if not (typ == 'temperature' and ('above' in msg or 'below' in msg)) or not ts:
            if current: grouped.append(current)
            current = None
            continue
        if not current:
            current = {'start_utc': ts, 'end_utc': ts, 'type': msg}
        elif current['type'] == msg:
            current['end_utc'] = ts
        else:
            grouped.append(current)
            current = {'start_utc': ts, 'end_utc': ts, 'type': msg}
    if current: grouped.append(current)
    return grouped

def format_timestamp_pacific(ts_str):
    try:
        dt = datetime.fromisoformat(ts_str.replace("Z", "+00:00")).astimezone(PACIFIC_TZ)
        return dt.strftime("%Y-%m-%d %H:%M:%S %Z")
    except Exception:
        return ts_str

def extract_unit_from_sensor(sensor_name):
    parts = sensor_name.split()
    for part in parts:
        if "-" in part and part.replace("-", "").isdigit():
            return part
    if "_" in sensor_name:
        return sensor_name.split("_")[0]
    return sensor_name

# Now do the lookup for that sensor
sensor_id = "TS00HGFPWC"  # your known sensor
sensor_name = None  # We'll fetch this

# Fetch sensor list to find the sensor name
sensors = safe_get_request(f"{BASE_URL}/sensors/all")
for s in sensors.get('data', {}).get('items', []):
    if s.get('sensor_id') == sensor_id:
        sensor_name = s.get('sensor_name', 'Unnamed')
        break

print(f"Sensor Name: {sensor_name}")

# Fetch recent alerts
alerts = get_recent_alerts(sensor_id)
print(f"Fetched {len(alerts)} alerts for sensor {sensor_name} ({sensor_id})\n")

print("Raw alerts:")
for alert in alerts:
    print(f"- Time: {alert.get('created')} | Type: {alert.get('event_type')} | Message: {alert.get('event_message')}")

# Group alerts by period
grouped_alerts = summarize_alert_periods(alerts)

# Connect to DB and match alert periods to incidents
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

results = []
for period in grouped_alerts:
    start_utc = period['start_utc']
    end_utc = period['end_utc']
    alert_type = period['type']

    try:
        t = datetime.fromisoformat(start_utc.replace("Z", "+00:00")).astimezone(PACIFIC_TZ)
    except Exception:
        t = None

    unit = extract_unit_from_sensor(sensor_name)

    match = {
        'status': '❌ No Match',
        'incident_datetime': None,
        'mitigation_efforts': None,
        'incident_id': None
    }
    if t and unit:
        w1, w2 = t - timedelta(minutes=15), t + timedelta(minutes=15)
        cursor.execute("""
            SELECT incident_datetime, outcome_text, incident_id
            FROM incident_reports
            WHERE unit = ? AND incident_datetime BETWEEN ? AND ?
            LIMIT 1
        """, (unit, w1.strftime('%Y-%m-%d %H:%M:%S'), w2.strftime('%Y-%m-%d %H:%M:%S')))
        row = cursor.fetchone()
        if row:
            match['status'] = '✅ Matched'
            match['incident_datetime'] = row[0]
            match['mitigation_efforts'] = row[1]
            match['incident_id'] = row[2]

    results.append({
        'start': format_timestamp_pacific(start_utc),
        'end': format_timestamp_pacific(end_utc),
        'type': alert_type,
        'match_status': match['status'],
        'match_time': match['incident_datetime'],
        'mitigation_efforts': match['mitigation_efforts'] or 'N/A',
        'incident_id': match['incident_id'] or 'N/A',
    })

conn.close()

# Show results
import pandas as pd
df = pd.DataFrame(results)
df

Sensor Name: 36-10_BLS
Fetched 10 alerts for sensor 36-10_BLS (TS00HGFPWC)

Raw alerts:
- Time: 2025-06-19 22:16:16Z | Type: temperature | Message: above 75 F (75.1 F)
- Time: 2025-06-19 21:51:20Z | Type: temperature | Message: above 75 F (75.8 F)
- Time: 2025-06-19 21:46:19Z | Type: temperature | Message: above 75 F (76.6 F)
- Time: 2025-06-19 21:41:19Z | Type: temperature | Message: above 75 F (76.2 F)
- Time: 2025-06-19 21:36:19Z | Type: temperature | Message: above 75 F (75.5 F)
- Time: 2025-06-19 21:31:19Z | Type: temperature | Message: above 75 F (75.4 F)
- Time: 2025-06-19 21:26:19Z | Type: temperature | Message: above 75 F (75.6 F)
- Time: 2025-06-19 21:21:19Z | Type: temperature | Message: above 75 F (75.9 F)
- Time: 2025-06-19 21:16:19Z | Type: temperature | Message: above 75 F (76.1 F)
- Time: 2025-06-19 21:11:19Z | Type: temperature | Message: above 75 F (76.7 F)


OperationalError: unable to open database file